# Analyse Bleu/Rouge ?

In [24]:
import numpy as np
import pandas as pd
from sacrebleu import BLEU
from rouge import Rouge 
import matplotlib.pyplot as plt
from reco_systems.filter import filter_df
import reco_systems.text_filtering as ft
from reco_systems.user_game_matrix import *
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# Construction de la BDD avec avis centrés
avis = pd.read_csv("database_cleaned/avis_clean.csv", index_col = 0)
min_reviews = 10
rev_filter = filter_df(avis, min_reviews)
rev_filter = rev_filter[rev_filter["Comment body"].str.split().apply(len) >= 10]
filtered_centrd_data, _= center_score(rev_filter)

def concat_inplace(df1, df2) -> pd.DataFrame:
    return pd.concat([df1,df2], ignore_index=True)

11201 10549
2252 2676
1957 2618
1947 2616
1946 2615
1945 2614


In [ ]:
from reco_systems.CF_knn import calc_similarity_matrix, get_KNN
from random import choice
rev_filter # data centrée

np.random.seed(45)
# Tirer un user au hasard 
# k-NN sur 20
# regarder le jeu qui a le plus de user commun
# generate user-game matrix

# Construction assoc
matrix_ratings, mask_ratings, users_table, games_table = get_matrix_user_game(filtered_centrd_data)
user_inds = filtered_centrd_data[['User id', 'Game id']].groupby("User id").count().reset_index()
user_inds.columns = ["User id", "Count reviews"]
assoc = users_table.to_frame().merge(user_inds, on="User id", how="left").reset_index()
assoc.columns = ["User index", "User id", "Count reviews"]
assoc.sort_values(by='Count reviews', ascending=False,inplace=True)

# Sélection aléatoire user
user_indices = assoc["User index"].to_numpy()


# Calcul matrice de similarité
sim_matrix = calc_similarity_matrix(matrix_ratings, mask_ratings, dist_type="cos")

bleu_rogue = pd.DataFrame(columns=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment','Type','Val'])
famous_users = assoc.head(20)["User index"].to_numpy()
print(famous_users)

for user in famous_users : 
# df : user_ref_id (le vrai), game_id évalué, voisin id, avis du voisin id, bleu, rouge
    #user = choice(user_indices)
    user_ind = assoc.loc[user,"User id"]
    print(user, " : ", user_ind)

    # k-nn
    neighbours = get_KNN(sim_matrix,20,user)
    print("Voisins : ", neighbours)

    neighbours_id = assoc[ np.isin(assoc['User index'],neighbours)]['User id'].to_numpy()
    neighbours_id
    neighbours_avis = filtered_centrd_data[(filtered_centrd_data['User id'].isin(neighbours_id)) | (filtered_centrd_data['User id'] == user_ind)]
    # Filtrer les jeux notés par user_ind
    user_games = filtered_centrd_data[filtered_centrd_data["User id"] == user_ind]["Game id"].unique()

    # Exclure les notes du user_ind -> garder uniquement les voisins !!!!!
    df_filtered = neighbours_avis[neighbours_avis["User id"] != user_ind]
    # Calculer la moyenne des notes pour chaque jeu
    game_avg_ratings = df_filtered.groupby("Game id")["Rating"].mean().reset_index()

    # Garder uniquement les jeux que user_ind a notés
    df_result = game_avg_ratings[game_avg_ratings["Game id"].isin(user_games)]
    # Renommer la colonne moyenne des notes
    df_result = df_result.rename(columns={"Rating": "Mean Rating"})
    df_result.sort_values('Mean Rating', ascending=False,inplace=True)

    best_game = df_result.loc[df_result["Mean Rating"].idxmax(), "Game id"]
    print("Jeu reco : ",int(best_game))
    df_result.head(10)

    neighbours_avis = neighbours_avis[neighbours_avis['Game id'] == best_game][['User id','Game id']]
    #df = ft.get_bleu_rouge(best_game,user_ind,assoc,neighbours,neighbours_avis)
    print("ID des voisins et du joueur : ", neighbours_avis['User id'].to_numpy())
    print("ID du ref : ",user_ind)
    ref_avis = rev_filter[(rev_filter['User id'] == user_ind) & (rev_filter['Game id'] == best_game)].index[0]
    print("Ref avis :", ref_avis)
    br = neighbours_avis.copy(True)
    br = br[br['User id'] != user_ind].reset_index()
    br.drop('Game id', axis = 1, inplace=True)
    br.columns = ['Avis id', 'User id']
    br.sort_values('Avis id', ascending=True)

    lem = pd.read_csv('generated_data/avis_lemmatized.csv',index_col=0)
    #lem_n = lem[np.isin(lem.index, br['Avis id'].to_numpy())]
    br = br.merge(lem,left_on='Avis id', right_index=True, how="inner") # merge left on right_index = True, left_on avis id
    #br['Lemma'] = lem_n['Comment'].to_numpy()
    #if(ref_avis not in lem.index) : continue
    ref_lem = lem.loc[ref_avis,'Comment']

    blue_scorer = BLEU()
    rouge_scorer = Rouge()

    br['Bleu'] = br['Comment'].apply(lambda x : blue_scorer.sentence_score(hypothesis=x,references=[ref_lem]).score/100)
    #br['Bleu nltk'] = br['Comment'].apply(lambda x : bleu_score.sentence_bleu(hypothesis=[list(x)],references=ref_lem))
    br['Rouge'] = br['Comment'].apply(lambda x : rouge_scorer.get_scores(hyps=x,refs=ref_lem)[0]["rouge-l"]["f"])
    br['User ref'] = [user_ind]*br.shape[0]
    br['Game rec'] = [best_game]*br.shape[0]
    br = br[['User ref', 'Game rec', 'Avis id', 'User id', 'Comment', 'Bleu', 'Rouge']]

    #bleu_rouge = pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")
    #bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")],ignore_index=True)
    bleu_rouge = concat_inplace(bleu_rogue, pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val"))
bleu_rouge

[121  40   1  46 117 541 633 416 956 145 681  47 902 346  49  54 235 601
 468 562]
121  :  208
Voisins :  [ 638  117  692  563  600    1  561  902 1261   40  145  541  833  976
 1061  468  505  362  610  238]
Jeu reco :  9476
ID des voisins et du joueur :  [ 208 1941 3628]
ID du ref :  208
Ref avis : 52253
40  :  83
Voisins :  [ 177  117  599  121  902  605 1061 1272  772  610  569   59  600  607
  302  681  618 1037 1460  692]
Jeu reco :  6000
ID des voisins et du joueur :  [ 208   83 2049]
ID du ref :  83
Ref avis : 78980


It is recommended to enable `effective_order` for sentence-level BLEU.
C:\Users\arlyc\AppData\Local\Temp\ipykernel_25732\3325687267.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")],ignore_index=True)


1  :  1
Voisins :  [ 332   46  121  145  379  629  101  123  102  362  167  407  245    3
  118 1485  692  464  704  197]
Jeu reco :  10118
ID des voisins et du joueur :  [208   1]
ID du ref :  1
Ref avis : 112494


It is recommended to enable `effective_order` for sentence-level BLEU.
C:\Users\arlyc\AppData\Local\Temp\ipykernel_25732\3325687267.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")],ignore_index=True)


46  :  91
Voisins :  [   1  176   29 1496  384 1345  693  198  780  104 1034  599  526  470
  117  737  874  522 1487  315]
Jeu reco :  1926
ID des voisins et du joueur :  [201  91]
ID du ref :  91
Ref avis : 131887


...

235  :  574
Voisins :  [ 636  541  642  828   49  933  607 1270 1854  605 1695  742  841  117
 1556  907  467  671  177  565]
Jeu reco :  5619
ID des voisins et du joueur :  [201 574]
ID du ref :  574
Ref avis : 124373


It is recommended to enable `effective_order` for sentence-level BLEU.
C:\Users\arlyc\AppData\Local\Temp\ipykernel_25732\3325687267.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")],ignore_index=True)


601  :  1900
Voisins :  [ 717  692  145 1141  629  681 1330  615 1767  287  612 1454  748  117
 1271 1739  283  846  214 1061]
Jeu reco :  2476
ID des voisins et du joueur :  [1900 2247]
ID du ref :  1900
Ref avis : 73734
468  :  1342
Voisins :  [ 817   80 1743  121 1560  302 1267 1929  117  599 1094 1089 1901  902
  825  854  571  166  770  772]
Jeu reco :  2360
ID des voisins et du joueur :  [1342 2385]
ID du ref :  1342
Ref avis : 92238
562  :  1816
Voisins :  [1447  885 1583 1618  722  795 1342 1482  604  817  608 1490  635  402
  627  914  954   51 1577 1726]
Jeu reco :  5423
ID des voisins et du joueur :  [1816 1926]
ID du ref :  1816
Ref avis : 8687


It is recommended to enable `effective_order` for sentence-level BLEU.
C:\Users\arlyc\AppData\Local\Temp\ipykernel_25732\3325687267.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")],ignore_index=True)


,User ref,Game rec,Avis id,User id,Comment,Type,Val
0,1816,5423,4212,1926,beau plonger encore innovation mecanique pearl...,Bleu,0.013489
1,1816,5423,4212,1926,beau plonger encore innovation mecanique pearl...,Rouge,0.074766


In [44]:
# Filtrer les jeux notés par user_ind
user_games = filtered_centrd_data[filtered_centrd_data["User id"] == user_ind]["Game id"].unique()

# Exclure les notes du user_ind -> garder uniquement les voisins !!!!!
df_filtered = neighbours_avis[neighbours_avis["User id"] != user_ind]
# Calculer la moyenne des notes pour chaque jeu
game_avg_ratings = df_filtered.groupby("Game id")["Rating"].mean().reset_index()

# Garder uniquement les jeux que user_ind a notés
df_result = game_avg_ratings[game_avg_ratings["Game id"].isin(user_games)]
# Renommer la colonne moyenne des notes
df_result = df_result.rename(columns={"Rating": "Mean Rating"})
df_result.sort_values('Mean Rating', ascending=False,inplace=True)

best_game = df_result.loc[df_result["Mean Rating"].idxmax(), "Game id"]
print(int(best_game))
df_result.head(10)

# Il faut filtrer avec l'avis sur le jeu de base

1077


,Game id,Mean Rating
22,1077,3.666667
272,10312,1.375000
103,3496,1.000000
184,6629,0.800000
170,6142,0.800000
192,6843,0.594737
199,7204,0.594737
32,1443,0.588235
223,8091,0.181818
30,1414,-0.166667


In [ ]:
from nltk.translate import bleu_score
from sacrebleu import sentence_bleu
from rouge import Rouge

neighbours_avis = neighbours_avis[neighbours_avis['Game id'] == best_game][['User id','Game id']]
#df = ft.get_bleu_rouge(best_game,user_ind,assoc,neighbours,neighbours_avis)
print(neighbours_avis['User id'].to_numpy())
print(user_ind)
ref_avis = neighbours_avis[neighbours_avis['User id'] == user_ind].index[0]
br = neighbours_avis.copy(True)
br = br[br['User id'] != user_ind].reset_index()
br.drop('Game id', axis = 1, inplace=True)
br.columns = ['Avis id', 'User id']
br.sort_values('Avis id', ascending=True)

lem = pd.read_csv('generated_data/avis_lemmatized.csv',index_col=0)
#lem_n = lem[np.isin(lem.index, br['Avis id'].to_numpy())]
br = br.merge(lem,left_on='Avis id', right_index=True, how="inner") # merge left on right_index = True, left_on avis id
#br['Lemma'] = lem_n['Comment'].to_numpy()
ref_lem = lem.loc[ref_avis,'Comment']

blue_scorer = BLEU()
rouge_scorer = Rouge()

br['Bleu'] = br['Comment'].apply(lambda x : blue_scorer.sentence_score(hypothesis=x,references=[ref_lem]).score/100)
#br['Bleu nltk'] = br['Comment'].apply(lambda x : bleu_score.sentence_bleu(hypothesis=[list(x)],references=ref_lem))
br['Rouge'] = br['Comment'].apply(lambda x : rouge_scorer.get_scores(hyps=x,refs=ref_lem)[0]["rouge-l"]["f"])
br['User ref'] = [user_ind]*br.shape[0]
br['Game rec'] = [best_game]*br.shape[0]
br = br[['User ref', 'Game rec', 'Avis id', 'User id', 'Comment', 'Bleu', 'Rouge']]

#bleu_rouge = pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")
bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")])

[2289 4735]
2289


It is recommended to enable `effective_order` for sentence-level BLEU.
C:\Users\arlyc\AppData\Local\Temp\ipykernel_25732\836748375.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bleu_rouge = pd.concat([bleu_rogue,pd.melt(br,id_vars=['User ref', 'Game rec', 'Avis id', 'User id', 'Comment'], value_vars=['Bleu', 'Rouge'],var_name="Type",value_name="Val")])


,User ref,Game rec,Avis id,User id,Comment,Type,Val
0,2289,1077,33809,4735,bluff enfumage retournement situation voici je...,Bleu,0.000345
1,2289,1077,33809,4735,bluff enfumage retournement situation voici je...,Rouge,0.058824
